In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats

import jb_helper_functions_prep
from jb_helper_functions_prep import create_enc

import prep_telco
from prep_telco import prep_telco_df

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import tree

In [4]:
df = prep_telco_df()
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,onlinesecurity_enc,onlinebackup_enc,deviceprotection_enc,techsupport_enc,streamingtv_enc,streamingmovies_enc,contract_enc,paperlessbilling_enc,paymentmethod_enc,churn_enc
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,0,2,0,0,0,0,0,1,2,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,2,0,2,0,0,0,1,0,3,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,2,2,0,0,0,0,0,1,3,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,2,0,2,2,0,0,1,0,0,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,0,0,0,0,0,0,0,1,2,1


In [5]:
train, test = train_test_split(df, test_size=.3, random_state=123, stratify=df[['churn_enc']])

In [22]:
print('Percent of non-churn: ' + str(train.churn_enc.value_counts()[0]/train.churn_enc.count()))
print('Percent of churn: ' + str(train.churn_enc.value_counts()[1]/train.churn_enc.count()))

Percent of non-churn: 0.7342543681430312
Percent of churn: 0.2657456318569687


In [6]:
y_train = train[['churn_enc']]
y_test = test[['churn_enc']]

In [7]:
df.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn',
       'gender_enc', 'partner_enc', 'dependents_enc', 'phoneservice_enc',
       'multiplelines_enc', 'internetservice_enc', 'onlinesecurity_enc',
       'onlinebackup_enc', 'deviceprotection_enc', 'techsupport_enc',
       'streamingtv_enc', 'streamingmovies_enc', 'contract_enc',
       'paperlessbilling_enc', 'paymentmethod_enc', 'churn_enc'],
      dtype='object')

In [27]:
X_train = train[['tenure', 'monthlycharges', 'internetservice_enc', 'techsupport_enc', 'contract_enc']]
X_test = test[['tenure', 'monthlycharges', 'internetservice_enc', 'techsupport_enc', 'contract_enc']]

In [28]:
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=123).fit(X_train, y_train)
y_pred = clf.predict(X_train)
y_pred_proba = clf.predict_proba(X_train)
print('Accuracy of Decision Tree classifier on training set: {:.6f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.795408


In [29]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.92      0.87      3614
           1       0.68      0.44      0.53      1308

   micro avg       0.80      0.80      0.80      4922
   macro avg       0.75      0.68      0.70      4922
weighted avg       0.78      0.80      0.78      4922



In [30]:
cm = pd.DataFrame(confusion_matrix(y_train, y_pred),
             columns=['Pred -', 'Pred +'], index=['Actual -', 'Actual +'])

cm

,Pred -,Pred +
Actual -,3337,277
Actual +,730,578
